In [4]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator     #Image data augumentation
from keras.layers import Dense,Activation,Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

In [2]:
def lr_schedule(epoch):
    lrate=0.001
    if epoch > 75: 
        lrate=0.0005
    if epoch > 100:
        lrate=0.0003
    return lrate

In [3]:
from tqdm import tqdm
import cv2
loc= 'A:/A_TEST/train/train'
import os
features = []
for i in tqdm(os.listdir(loc)):
    f = cv2.imread(os.path.join(loc,i),0)
    fr = cv2.resize(f,(32,32))
    features.append(fr)

100%|██████████| 25000/25000 [15:44<00:00, 26.46it/s]  


In [6]:
import numpy as np
X=np.array(features)
X.shape

(25000, 32, 32)

In [5]:
label = []
for i in os.listdir(loc):
    if(i.split('.')[0] == 'cat'):
        label.append(0)
    elif(i.split('.')[0] == 'dog'):
        label.append(1)
    else:
        label.append(2)

In [7]:
Y=np.array(label)
Y.shape

(25000,)

In [8]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y)

In [9]:
xtrain.shape
xtest.shape

(6250, 32, 32)

In [10]:
#Normalization of input data
mean = np.mean(xtrain)
std = np.std(xtrain)
xtrain = (xtrain - mean)/std
xtest = (xtest-mean)/std

In [11]:
#Categorazation of o/p value
ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)

In [12]:
xtrain=xtrain.reshape(18750,32,32,1)
xtest=xtest.reshape(6250, 32, 32,1)
ytrain.shape

(18750, 2)

In [13]:
#Build CNN Model
wd = 0.0001
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer = regularizers.l2(wd), input_shape=xtrain.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer = regularizers.l2(wd)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer = regularizers.l2(wd)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer = regularizers.l2(wd)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer = regularizers.l2(wd)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer = regularizers.l2(wd)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(2,activation='softmax'))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [ ]:
opt_rms = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=opt_rms,metrics=['accuracy'])
model.fit(xtrain,ytrain,epochs=10,
          validation_data=(xtest,ytest),callbacks=[LearningRateScheduler(lr_schedule)])

Train on 18750 samples, validate on 6250 samples
Epoch 1/10
18750/18750 [==============================] - ETA: 42:48 - loss: 1.7496 - acc: 0.5000 - ETA: 27:44 - loss: 3.3293 - acc: 0.5000 - ETA: 22:45 - loss: 3.4893 - acc: 0.5000 - ETA: 20:29 - loss: 3.1611 - acc: 0.4766 - ETA: 19:03 - loss: 3.1380 - acc: 0.4875 - ETA: 18:10 - loss: 2.8405 - acc: 0.5052 - ETA: 17:24 - loss: 2.6721 - acc: 0.4911

In [0]:
import matplotlib.pyplot as plt
plt.imshow(xtest[345])
plt.show()

In [0]:
np.argmax(model.predict(xtest[345].reshape(1,32,32,3)))

In [0]:
np.argmax(ytest[345])